In [120]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'


In [121]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [122]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [123]:
#usage
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
print(f"{BOLD}POWERS IN DBM{RESET}")
pop_size_dbm = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size_dbm)

POWERS IN DBM
[[2 0 4 0]
 [0 4 2 4]
 [4 0 0 2]
 [2 4 4 2]
 [4 2 4 2]
 [0 2 0 0]
 [2 4 4 4]
 [4 2 4 4]]


In [124]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [125]:
#usage
print(f"{BOLD}POWERS IN LINEAR{RESET}")
linear_pop=calculate_linear_trans_power(pop_size_dbm)
print(linear_pop)

POWERS IN LINEAR
[[0.00158489 0.001      0.00251189 0.001     ]
 [0.001      0.00251189 0.00158489 0.00251189]
 [0.00251189 0.001      0.001      0.00158489]
 [0.00158489 0.00251189 0.00251189 0.00158489]
 [0.00251189 0.00158489 0.00251189 0.00158489]
 [0.001      0.00158489 0.001      0.001     ]
 [0.00158489 0.00251189 0.00251189 0.00251189]
 [0.00251189 0.00158489 0.00251189 0.00251189]]


In [126]:
def sum_linear_powers(array):
    return np.sum(array, axis=1)

In [127]:
# usage
print(f"{BOLD}SUM OF LINEAR POWERS BY ROW{RESET}")
row_sums = sum_linear_powers(linear_pop)
print(*row_sums, sep='\n')

SUM OF LINEAR POWERS BY ROW
0.006096779623970694
0.007608666055480274
0.006096779623970694
0.008193559247941387
0.008193559247941387
0.0045848931924611135
0.009120552486989855
0.009120552486989855


In [128]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

Linear noise: [3.98107171e-15]


In [129]:
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [130]:
#Usage 
print(f"{BOLD}BITRATES(formula){RESET}")
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

BITRATES(formula)
[[3523086.17671602 2232475.21650449 4272652.02625749 2577774.03041514]
 [2930294.44975843 3363516.1582802  3651275.96911754 3753232.64345308]
 [4140399.83799027 2232475.21650449 3052533.55366803 3150104.99872635]
 [3523086.17671602 3363516.1582802  4272652.02625749 3150104.99872635]
 [4140399.83799027 2778870.2310227  4272652.02625749 3150104.99872635]
 [2930294.44975843 2778870.2310227  3052533.55366803 2577774.03041514]
 [3523086.17671602 3363516.1582802  4272652.02625749 3753232.64345308]
 [4140399.83799027 2778870.2310227  4272652.02625749 3753232.64345308]]


In [131]:
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [132]:
# USAGE
print(f"{BOLD}SUM OF BITRATES{RESET}")
row_sum_bitrate = row_wise_sums(bitrate)
print(*row_sum_bitrate, sep='\n')


SUM OF BITRATES
12605987.449893143
13698319.220609253
12575513.606889138
14309359.359980067
14342027.093996806
11339472.264864285
14912487.0047068
14945154.738723539


In [133]:
def sort_row_sums(array):
    return np.sort(array)


In [134]:
print(f"{BOLD}SORTED LINEAR POWER IN INCREASING ORDER{RESET}")
sorted_array_linear_power = sort_row_sums(row_sums)
print(*sorted_array_linear_power, sep='\n')  # Output: [1 3 5 7 9]

SORTED LINEAR POWER IN INCREASING ORDER
0.0045848931924611135
0.006096779623970694
0.006096779623970694
0.007608666055480274
0.008193559247941387
0.008193559247941387
0.009120552486989855
0.009120552486989855


In [135]:
def sort_row_sums(array):
    return np.sort(array)

In [136]:
print(f"{BOLD}CORRESPONDING BITRATES{RESET}")
sorted_bitrates = sorted_bitrates_by_linear_power(linear_pop, SUM_GAIN, linear_noise)
print(*sorted_bitrates, sep='\n')


CORRESPONDING BITRATES
11339472.264864285
12605987.449893143
12575513.606889138
13698319.220609253
14309359.359980067
14342027.093996806
14912487.0047068
14945154.738723539


In [138]:
def get_sorted_dbm(sorted_linear_power, dbm_population):
    linear_pop = calculate_linear_trans_power(dbm_population)
    row_sums = sum_linear_powers(linear_pop)
    
    sorted_dbm = []
    for linear_power in sorted_linear_power:
        index = np.where(np.isclose(row_sums, linear_power))[0][0]
        sorted_dbm.append(dbm_population[index])
    return np.array(sorted_dbm)

# Usage
sorted_dbm = get_sorted_dbm(sorted_array_linear_power, pop_size_dbm)

print(f"{BOLD}CORRESPONDING POWERS IN DB TO LINEAR POWER{RESET}")
print(sorted_dbm)


CORRESPONDING POWERS IN DB TO LINEAR POWER
[[0 2 0 0]
 [2 0 4 0]
 [2 0 4 0]
 [0 4 2 4]
 [2 4 4 2]
 [2 4 4 2]
 [2 4 4 4]
 [2 4 4 4]]


In [139]:
def select_first_half(array):
    midpoint = len(array) // 2
    return array[:midpoint]


In [140]:
#usage1
first_half_linear_power_increasing = select_first_half(sorted_array_linear_power)
print(f"{BOLD}TOP HALF OF SORTED ROW SUMS OF LINEAR POWER{RESET}")
print(*first_half_linear_power_increasing, sep='\n') 
 

TOP HALF OF SORTED ROW SUMS OF LINEAR POWER
0.0045848931924611135
0.006096779623970694
0.006096779623970694
0.007608666055480274


In [147]:
def upper_half_bitrates(sorted_bitrates):
    # Calculate the index where to split the list
    split_index = len(sorted_bitrates) // 2

    # Get the upper half of the sorted bitrates
    upper_half_bitrates = sorted_bitrates[:split_index]

    return upper_half_bitrates


In [148]:
# sorted_bitrates = sorted_bitrates_by_linear_power(linear_pop, SUM_GAIN, linear_noise)
print(f"{BOLD}UPPER HALF OF BITRATES CORRESPONDING TO TOP HALF OF LINEAR POWER{RESET}")
upper_half_bitrates = upper_half_bitrates(sorted_bitrates)
print(*upper_half_bitrates, sep='\n')


UPPER HALF OF BITRATES CORRESPONDING TO TOP HALF OF LINEAR POWER
11339472.264864285
12605987.449893143
12575513.606889138
13698319.220609253


In [149]:
def get_top_half_dbm(pop_size_dbm, row_sums, first_half_linear_power_increasing):
    # Get the indices of the top half of the sorted row sums of linear power
    top_half_linear_power_indices = [np.where(row_sums == value)[0][0] for value in first_half_linear_power_increasing]

    # Get the power allocations in dBm corresponding to the top half of the sorted row sums of linear power
    top_half_linear_power_dbm = pop_size_dbm[top_half_linear_power_indices]

    return top_half_linear_power_dbm


In [150]:
# Usage
top_half_dbm = get_top_half_dbm(pop_size_dbm, row_sums, first_half_linear_power_increasing)
print(f"{BOLD}TOP HALF OF POWER ALLOCATIONS IN DBM CORRESPONDING TO SORTED ROW SUMS OF LINEAR POWER(INITIAL POPULATION){RESET}")
print(top_half_dbm)


TOP HALF OF POWER ALLOCATIONS IN DBM CORRESPONDING TO SORTED ROW SUMS OF LINEAR POWER(INITIAL POPULATION)
[[0 2 0 0]
 [2 0 4 0]
 [2 0 4 0]
 [0 4 2 4]]


In [151]:
def simple_single_point_crossover(population):
    num_offspring = len(population)
    offspring = []

    for i in range(0, num_offspring, 2):
        # Select two parent solutions sequentially
        parent1, parent2 = population[i], population[i + 1]

        # Randomly choose a crossover point
        crossover_point = random.randint(1, len(parent1) - 1)

        # Create new offspring by swapping the genes of the parents at the crossover point
        offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

        offspring.extend([offspring1, offspring2])

    return np.array(offspring)

# Usage
offspring = simple_single_point_crossover(top_half_dbm)

print(f"{BOLD}OFFSPRING AFTER SINGLE-POINT CROSSOVER{RESET}")
print(offspring)


OFFSPRING AFTER SINGLE-POINT CROSSOVER
[[0 0 4 0]
 [2 2 0 0]
 [2 0 4 4]
 [0 4 2 0]]


In [152]:
def mutation(offspring, powers, mutation_rate=1.0):
    mutated_offspring = offspring.copy()

    for i in range(len(offspring)):
        if random.random() < mutation_rate:
            # Randomly select a gene index for mutation
            gene_index = random.randint(0, len(offspring[i]) - 1)

            # Choose a new power value randomly, different from the current gene value
            new_power = random.choice([p for p in powers if p != offspring[i][gene_index]])

            # Replace the gene with the new power value
            mutated_offspring[i][gene_index] = new_power

    return mutated_offspring



In [153]:
# Usage
mutated_offspring = mutation(offspring, powers)
print(f"{BOLD}MUTATED OFFSPRING{RESET}")
print(mutated_offspring)

MUTATED OFFSPRING
[[0 0 2 0]
 [2 4 0 0]
 [2 0 2 4]
 [0 4 2 4]]


In [154]:
def combine_initial_and_mutated(initial_population, mutated_offspring):
    return np.concatenate((initial_population, mutated_offspring), axis=0)

# Usage
combined_population = combine_initial_and_mutated(top_half_dbm, mutated_offspring)
print(f"{BOLD}COMBINED POPULATION{RESET}")
print(combined_population)


COMBINED POPULATION
[[0 2 0 0]
 [2 0 4 0]
 [2 0 4 0]
 [0 4 2 4]
 [0 0 2 0]
 [2 4 0 0]
 [2 0 2 4]
 [0 4 2 4]]
